In [6]:
!pip install py2neo


  Using cached py2neo-2021.2.4-py2.py3-none-any.whl.metadata (9.9 kB)
Using cached py2neo-2021.2.4-py2.py3-none-any.whl (177 kB)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
from py2neo import Graph, Node

In [23]:

driver = Graph("bolt://localhost:7687", auth=("neo4j", "Pranjali@1357"))


In [5]:
!pip install hnswlib



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install git+https://github.com/nmslib/hnswlib.git



  Cloning https://github.com/nmslib/hnswlib.git to c:\users\pranjali\appdata\local\temp\pip-req-build-xxxjid45
  Resolved https://github.com/nmslib/hnswlib.git to commit c1b9b79af3d10c6ee7b5d0afa1ce851ae975254c
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp311-cp311-win_amd64.whl size=150797 sha256=bf2a2d795886be6ffed7d91db0b7c05d0a84f3180cfcb78825d4c6671eb51380
  Stored in directory: C:\Users\Pranjali\AppData\Local\Temp\pip-ephem-wheel-cache-_p_eh1qs\wheels\ac\bf\09\807282311a5fc440fc3da211a3e5f42a71c1497b70dcdfccfd
Successfully built hnswlib


  Running command git clone --filter=blob:none --quiet https://github.com/nmslib/hnswlib.git 'C:\Users\Pranjali\AppData\Local\Temp\pip-req-build-xxxjid45'

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: C:\Users\Pranjali\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [19]:
def create_hnsw_index(vectors):
    dim = vectors.shape[1]  # Dimension of the vectors
    index = hnswlib.Index(space='l2', dim=dim)  # L2 distance (Euclidean distance)
    index.init_index(max_elements=vectors.shape[0], ef_construction=200, M=16)
    index.add_items(vectors)
    return index

In [11]:
import hnswlib
import numpy as np


In [14]:
from neo4j import GraphDatabase


In [10]:
pip install neo4j


   ---------------------------------------- 0.0/301.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/301.7 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/301.7 kB 1.4 MB/s eta 0:00:01
   ----- --------------------------------- 41.0/301.7 kB 495.5 kB/s eta 0:00:01
   ----------- --------------------------- 92.2/301.7 kB 751.6 kB/s eta 0:00:01
   ------------------ ------------------- 143.4/301.7 kB 853.3 kB/s eta 0:00:01
   ------------------------ ------------- 194.6/301.7 kB 908.0 kB/s eta 0:00:01
   ------------------------------------ --- 276.5/301.7 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 301.7/301.7 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: C:\Users\Pranjali\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [24]:
def create_knn_graph(nodes, k=5):
    # Step 3a: Get the vector representations of the nodes (assuming each node has a 'vector' property)
    vectors = []
    node_ids = []
    
    with driver.session() as session:
        # Retrieve nodes with their vector property
        result = session.run("MATCH (n) WHERE exists(n.vector) RETURN ID(n) AS node_id, n.vector AS vector")
        
        for record in result:
            node_ids.append(record['node_id'])
            vectors.append(np.array(record['vector']))
    
    vectors = np.array(vectors)

    # Step 3b: Compute the k-NN using HNSW
    index = create_hnsw_index(vectors)
    labels, distances = index.knn_query(vectors, k=k)  # Get k nearest neighbors

    # Step 3c: Store relationships in Neo4j
    with driver.session() as session:
        for i, node_id in enumerate(node_ids):
            for j, neighbor in enumerate(labels[i]):
                distance = distances[i][j]
                if node_id != neighbor:  # Avoid self-relationships
                    # Create relationship between the node and its neighbor
                    session.run(
                        "MATCH (a), (b) WHERE id(a) = $node_id AND id(b) = $neighbor_id "
                        "MERGE (a)-[r:DISTANT_FROM {distance: $distance}] -> (b)",
                        node_id=node_id,
                        neighbor_id=node_ids[neighbor],
                        distance=distance
                    )

    print(f"Created k-NN graph with {k} neighbors for each node.")

# Example usage:
create_knn_graph(nodes=None, k=5) 

AttributeError: 'Graph' object has no attribute 'session'

In [8]:
import pandas as pd

In [60]:
uri = "bolt://localhost:7687"  # Adjust with your Neo4j URI
username = "neo4j"  # Replace with your username
password = "Pranjali@1357"  # Replace with your password

driver = GraphDatabase.driver(uri, auth=(username, password))

# Load your data (assuming CSV format with given columns)
data = pd.read_csv("creditcard_fraud20.csv")  # Adjust with your data path
vectors = np.array(data["vector"].apply(eval).to_list())  # Parse stringified lists

# Parameters
dimension = 28  # Size of the vector
num_elements = len(data)
k = 3  # Number of neighbors
distance_function = "l2"  # Euclidean distance

# Initialize hnswlib index
p = hnswlib.Index(space=distance_function, dim=dimension)
p.init_index(max_elements=num_elements, ef_construction=200, M=16)
p.add_items(vectors)

# Perform k-NN search
labels, distances = p.knn_query(vectors, k=k)


In [19]:
def create_relationships(tx, id1, id2, distance):
    query = """
    MATCH (a:Transaction {id: $id1}), (b:Transaction {id: $id2})
    MERGE (a)-[:DISTANT_FROM {weight: $distance}]->(b)
    """
    tx.run(query, id1=id1, id2=id2, distance=distance)

In [20]:
with driver.session() as session:
    for i, node_id in enumerate(data["ID"]):
        for neighbor_idx, dist in zip(labels[i], distances[i]):
            neighbor_id = data.iloc[neighbor_idx]["ID"]
            session.execute_write(create_relationships, node_id, neighbor_id, dist)

In [48]:

# Generate Average Distance CSV for Fraudulent Transactions
def export_csv(tx, is_fraud, file_name):
    query = f"""
    CALL apoc.export.csv.query(
        'MATCH (t:Transaction)-[r:DISTANT_FROM]->(other: Transaction)
        WHERE t.isFraud = {is_fraud}
        RETURN t.transactionid as transactionid, avg(r.weight) as weight',
        null, {{stream:true}})
    YIELD data
    RETURN data
    """
    tx.run(query)


with driver.session() as session:
    session.write_transaction(export_csv, 0, "non_fraud.csv")
    session.write_transaction(export_csv, 1, "fraud.csv")
   

print("k-NN graph created and CSVs exported.")

k-NN graph created and CSVs exported.


C:\Users\Pranjali\AppData\Local\Temp\ipykernel_21740\2986551595.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(export_csv, 0, "non_fraud.csv")
C:\Users\Pranjali\AppData\Local\Temp\ipykernel_21740\2986551595.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(export_csv, 1, "fraud.csv")


In [49]:
import os
print("Current working directory:", os.getcwd())


Current working directory: C:\Users\Pranjali


In [59]:
CALL apoc.export.csv.query(
  'MATCH (t:Transaction)-[r:DISTANT_FROM]->(other:Transaction)
   WHERE t.isFraud = 1
   RETURN t.transactionid AS transactionid, avg(r.weight) AS weight',
  'fraud.csv',
  {batchSize:1000, delim: '\t', quotes: false, format: 'plain', header: true}
)



In [56]:
def export_csv(tx, is_fraud, file_name):
    query = f"""
   CALL apoc.export.cv.query'MATCH (t:Transaction)-[r: DISTANT_FROM]->
→ (other:Transaction)
WHERE t.isFraud = 0
RETURN t.transactionid as transactionld, avg(r.weight) as weight,
- 'noFraud.csv,
{batchSize:1000}, delim: 'It', quotes: false, format: 'plain', header:true})
    """
    tx.run(query)
# # Export average distance for non-fraudulent transactions
# with driver.session() as session:
#     session.write_transaction(export_csv, 0, "non_fraud.csv")
# # Export average distance for fraudulent transactions
# with driver.session() as session:
#     session.write_transaction(export_csv, 1, "fraud.csv")



print("k-NN graph created and CSVs exported.")

SyntaxError: f-string: single '}' is not allowed (769123339.py, line 9)

In [65]:
import hnswlib
import pandas as pd
from neo4j import GraphDatabase

# Load the data
data_path = "creditcard_fraud20.csv"  # Path to your CSV file
df = pd.read_csv(data_path)

# Initialize HNSW index
dimension = 28  # Dimension of the vectors
p = hnswlib.Index(space='l2', dim=dimension)
p.init_index(max_elements=len(df), ef_construction=200, M=16)

# Add vectors to HNSW index
vectors = df['vector'].apply(eval).tolist()  # Convert stringified lists to actual lists
p.add_items(vectors)

# Query for k-nearest neighbors
k = 3
neighbors, distances = p.knn_query(vectors, k=k)

# Connect to Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "Pranjali@1357"
driver = GraphDatabase.driver(uri, auth=(username, password))


def create_relationship(tx, t1, t2, dist):
    print(f"Creating relationship between {t1} and {t2} with distance {dist}")
    query = """
    MATCH (a:Transaction {ID: $t1}), (b:Transaction {ID: $t2})
    CREATE (a)-[:DISTANT_FROM {weight: $dist}]->(b)
    """
    tx.run(query, t1=t1, t2=t2, dist=dist)


# Create k-NN relationships
with driver.session() as session:
    for i, (nbrs, dists) in enumerate(zip(neighbors, distances)):
        for nbr, dist in zip(nbrs, dists):
            session.write_transaction(create_relationship, df.loc[i, "ID"], df.loc[nbr, "ID"], dist)

driver.close()


C:\Users\Pranjali\AppData\Local\Temp\ipykernel_21740\3065621319.py:42: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, df.loc[i, "ID"], df.loc[nbr, "ID"], dist)


Creating relationship between 130999 and 130999 with distance 0.0
Creating relationship between 130999 and 197884 with distance 7.768036365509033
Creating relationship between 130999 and 263151 with distance 8.111936569213867
Creating relationship between 66755 and 66755 with distance 0.0
Creating relationship between 66755 and 254763 with distance 60.33540725708008
Creating relationship between 66755 and 250913 with distance 72.39511108398438
Creating relationship between 275102 and 275102 with distance 0.0
Creating relationship between 275102 and 195833 with distance 0.35502558946609497
Creating relationship between 275102 and 227695 with distance 0.3742518424987793
Creating relationship between 167029 and 167029 with distance 0.0
Creating relationship between 167029 and 217636 with distance 4.693508148193359
Creating relationship between 167029 and 196580 with distance 4.831997871398926
Creating relationship between 15878 and 15878 with distance 0.0
Creating relationship between 158

In [62]:
def test_connection(driver):
    try:
        with driver.session() as session:
            result = session.run("RETURN 1")
            if result.single() is not None:
                print("Connection is fine!")
            else:
                print("Connection test failed.")
    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        driver.close()

# Run the connection test
test_connection(driver)

C:\Users\Pranjali\AppData\Local\Temp\ipykernel_21740\2709433165.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Connection is fine!


In [64]:
import requests

def check_neo4j_status():
    try:
        response = requests.get('http://localhost:7474')
        if response.status_code == 200:
            print("Neo4j server is up and running.")
        else:
            print(f"Server returned status code {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

check_neo4j_status()


Error: No connection adapters were found for 'bolt://localhost:7687'


In [67]:
with driver.session() as session:
    session.execute_write(create_relationship, 221881, 280051, 0.11048223078250885)


Creating relationship between 221881 and 280051 with distance 0.11048223078250885


C:\Users\Pranjali\AppData\Local\Temp\ipykernel_21740\191787662.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [85]:
from neo4j import GraphDatabase
data_path = "creditcard_fraud20.csv"  # Path to your CSV file
df = pd.read_csv(data_path)

# Initialize HNSW index
dimension = 28  # Dimension of the vectors
p = hnswlib.Index(space='l2', dim=dimension)
p.init_index(max_elements=len(df), ef_construction=200, M=16)

# Add vectors to HNSW index
vectors = df['vector'].apply(eval).tolist()  # Convert stringified lists to actual lists
p.add_items(vectors)

# Query for k-nearest neighbors
k = 3
neighbors, distances = p.knn_query(vectors, k=k)

# Create a driver instance
uri = "bolt://localhost:7687"  # Adjust your URI if needed
username = "neo4j"  # Your username
password = "Pranjali@1357"  # Your password

# Initialize the driver (ensure it's open)
driver = GraphDatabase.driver(uri, auth=(username, password))

def create_relationship(tx, t1, t2, dist):
    print(f"Creating relationship between {t1} and {t2} with distance {dist}")
    query = """
    MATCH (a:Transaction {ID: $t1}), (b:Transaction {ID: $t2})
    MERGE (a)-[:DISTANT_FROM {weight: $dist}]->(b)
    """
    tx.run(query, t1=t1, t2=t2, dist=dist)


try:
    with driver.session() as session:
         for i, (nbrs, dists) in enumerate(zip(neighbors, distances)):
            for nbr, dist in zip(nbrs, dists):
                if df.loc[i, "ID"] != df.loc[nbr, "ID"]:
                  session.execute_write(create_relationship, df.loc[i, "ID"], df.loc[nbr, "ID"], dist)
except Exception as e:
    print(f"Error occurred: {e}")




Creating relationship between 130999 and 197884 with distance 7.768036365509033
Creating relationship between 130999 and 263151 with distance 8.111936569213867
Creating relationship between 66755 and 254763 with distance 60.33540725708008
Creating relationship between 66755 and 250913 with distance 72.39511108398438
Creating relationship between 275102 and 195833 with distance 0.35502558946609497
Creating relationship between 275102 and 227695 with distance 0.3742518424987793
Creating relationship between 167029 and 217636 with distance 4.693508148193359
Creating relationship between 167029 and 196580 with distance 4.831997871398926
Creating relationship between 15878 and 86847 with distance 3.2973361015319824
Creating relationship between 15878 and 51285 with distance 3.331841468811035
Creating relationship between 116162 and 58838 with distance 3.1489932537078857
Creating relationship between 116162 and 138014 with distance 4.2512383460998535
Creating relationship between 85951 and 4

In [80]:
def create_relationship(tx, t1, t2, dist):
    print(f"Creating relationship between {t1} and {t2} with distance {dist}")
    query = """
    MATCH (a:Transaction {ID: $t1}), (b:Transaction {ID: $t2})
    MERGE (a)-[:DISTANT_FROM {weight: $dist}]->(b)
    """
    tx.run(query, t1=t1, t2=t2, dist=dist)

try:
    with driver.session() as session:
                  session.execute_write(create_relationship, 99932,141857,7.557013511657715)
except Exception as e:
    print(f"Error occurred: {e}")


Creating relationship between 99932 and 141857 with distance 7.557013511657715


In [83]:
def create_relationship(tx, t1, t2, dist):
    print(f"Creating relationship between {t1} and {t2} with distance {dist}")
    query = """
    MATCH (a:Transaction {ID: $t1}), (b:Transaction {ID: $t2})
    MERGE (a)-[:DISTANT_FROM {weight: $dist}]->(b)
    """
    result = tx.run(query, t1=t1, t2=t2, dist=dist)
    print(f"Transaction result: {result}")

try:
    with driver.session() as session:
        session.execute_write(create_relationship, 99932, 141857, 7.557013511657715)
except Exception as e:
    print(f"Error occurred: {e}")


Creating relationship between 99932 and 141857 with distance 7.557013511657715
Transaction result: <neo4j._sync.work.result.Result object at 0x000001ED507299D0>


In [90]:
def export_fraud_transactions(tx):
    query = """
    CALL apoc.export.csv.query(
        'MATCH (t:Transaction)-[r:DISTANT_FROM]->(other:Transaction)
         WHERE t.isFraud = TRUE
         RETURN t.ID AS transactionid, avg(r.weight) AS weight',
        null,
        {stream: true, batchSize: 5000, delim: ',', quotes: false, format: 'plain', header: true}
    )
    YIELD data
    RETURN data
    """
    # Run the query
    result = tx.run(query)

    # Stream the data
    for record in result:
        # Process each record (e.g., print it or save it to a file)
        print(record['data'])  # This will print the streamed data

# Execute the query within a session
try:
    with driver.session() as session:
        session.read_transaction(export_fraud_transactions)
except Exception as e:
    print(f"Error occurred: {e}")

C:\Users\Pranjali\AppData\Local\Temp\ipykernel_21740\2771762152.py:24: DeprecationWarning: read_transaction has been renamed to execute_read
  session.read_transaction(export_fraud_transactions)


transactionid,weight
8843,498.441162109375
17408,2422.6456705729165
42675,1774.4990234375
12370,1961.8603515625
143334,733.1575724283854
149358,103.88893508911133
74795,56.34907150268555
15226,1592.22119140625



In [94]:
import pandas as pd
import requests

# Load JSON data from local URL
url = 'http://localhost:8889/lab/tree/records.json'
response = requests.get(url)
json_data = response.json()

# Convert JSON data to DataFrame
df = pd.DataFrame(json_data)

# Save DataFrame to CSV
df.to_csv('output.csv', index=False)

print("JSON data has been converted to CSV and saved as 'output.csv'")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [97]:
import pandas as pd

# Path to the text file
txt_file_path = 'tx.txt'  # Replace with your file's name

# Read the text file and create a DataFrame
try:
    # Read the text file
    df = pd.read_csv(txt_file_path, header=None, names=['ID', 'weight'])
    
    # Save to CSV
    csv_file_path = 'output.csv'
    df.to_csv(csv_file_path, index=False)
    
    print(f"Text file has been successfully converted to CSV and saved as {csv_file_path}.")
except Exception as e:
    print(f"An error occurred: {e}")


Text file has been successfully converted to CSV and saved as output.csv.


In [2]:
!pip install umap-learn


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\tqdm.exe' -> 'C:\\Python312\\Scripts\\tqdm.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached umap_learn-0.5.7-py3-none-any.whl.metadata (21 kB)
  Using cached numba-0.60.0-cp312-cp312-win_amd64.whl.metadata (2.8 kB)
  Using cached pynndescent-0.5.13-py3-none-any.whl.metadata (6.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached llvmlite-0.43.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
Using cached umap_learn-0.5.7-py3-none-any.whl (88 kB)
Using cached numba-0.60.0-cp312-cp312-win_amd64.whl (2.7 MB)
Using cached pynndescent-0.5.13-py3-none-any.whl (56 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached llvmlite-0.43.0-cp312-cp312-win_amd64.whl (28.1 MB)
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)


In [5]:
from py2neo import Graph


import pandas as pd
import numpy as np

# Step 1: Connect to the Neo4j server
# Replace "your_password" with the actual password you set for the Neo4j user.
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "Pranjali@1357"

# Connecting to the Neo4j graph
graph = Graph(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
print("Connected to Neo4j database.")

# Step 2: Sanity check for the database connection
databases = graph.run("SHOW DATABASES").data()
print("Available databases:", databases)

# Step 3: Create a virtual subgraph for GDS
# Creating a subgraph with Article nodes and SIMILAR_TO relationships
GDS_GRAPH_NAME = "similarity_subgraph"

# Drop existing GDS graph if it exists
graph.run(f"CALL gds.graph.drop('{GDS_GRAPH_NAME}', false) YIELD graphName;")

# Create the new GDS graph
create_graph_query = f"""
CALL gds.graph.project(
    '{GDS_GRAPH_NAME}',
    'Article',
    {{
        SIMILAR_TO: {{
            type: 'SIMILAR_TO',
            properties: ['weight']
        }}
    }}
)
"""
graph.run(create_graph_query)
print(f"Created GDS graph: {GDS_GRAPH_NAME}")

Connected to Neo4j database.
Available databases: [{'name': 'av-graph', 'type': 'standard', 'aliases': [], 'access': 'read-write', 'address': 'localhost:7687', 'role': 'primary', 'writer': True, 'requestedStatus': 'offline', 'currentStatus': 'unknown', 'statusMessage': '', 'default': False, 'home': False, 'constituents': []}, {'name': 'neo4j', 'type': 'standard', 'aliases': [], 'access': 'read-write', 'address': 'localhost:7687', 'role': 'primary', 'writer': True, 'requestedStatus': 'offline', 'currentStatus': 'unknown', 'statusMessage': '', 'default': False, 'home': False, 'constituents': []}, {'name': 'new-graph', 'type': 'standard', 'aliases': [], 'access': 'read-write', 'address': 'localhost:7687', 'role': 'primary', 'writer': True, 'requestedStatus': 'online', 'currentStatus': 'online', 'statusMessage': '', 'default': True, 'home': True, 'constituents': []}, {'name': 'system', 'type': 'system', 'aliases': [], 'access': 'read-write', 'address': 'localhost:7687', 'role': 'primary', 

ClientError: [Procedure.ProcedureNotFound] There is no procedure with the name `gds.graph.drop` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.

In [4]:
pip show umap-learn


Note: you may need to restart the kernel to use updated packages.


In [8]:
print(df)

NameError: name 'df' is not defined